Seguridad
=========

Ingeniería Web, 2019

Cuánto hay para saber de seguridad?
=================================

**Mucho**.

Es prácticamente un mundo aparte. Sus propias prácticas, libros, conferencias, carreras, herramientas, etc.

Y cuánto **tengo** que saber de seguridad?
=================================

Lo más que puedas. Nunca está de más.

En esta clase vamos a ver solo algunos temas puntuales, **importantes, pero básicos**.

Sobre soluciones caseras...
=================================

Escenario típico:

"Tengo que guardar la contraseña de forma segura..."

"Ya se! Podría programar algo que la invierta, le sume 3 letras a cada caracter, y después..."

![](./no.jpg)

No. Nunca. NO INVENTEN.

**Las soluciones caseras nunca son seguras.**

Algoritmos diseñados por especialistas en criptografía con décadas de experiencia estudiando y diseñando algoritmos seguros, fueron rotos. Romper tu solución casera va a ser infinitamente más fácil que eso.

**Los programadores no somos expertos en seguridad.**

Las cosas diseñadas por gente no experta en seguridad, son super vulnerables.

Los programadores tenemos que hacer lo que los expertos en seguridad diseñan/recomiendan.

Seguridad por oscuridad
====================

- "Nadie puede desencriptar este paquete de red, porque nadie sabe **cómo** lo encripté".
- "Nadie puede acceder a este servidor, porque nadie sabe cómo lo configuré".
- "Nadie puede crackear este programa, porque nadie sabe cómo valido que sea original".
- ...

Confiar en que algo no es seguro, porque otros no saben **cómo** fue hecho seguro.

Por lo general, **es una muy mala idea**. La práctica mostró siempre que si es importante, se termina descubriendo.

Una buena práctica o algoritmo de seguridad, es seguro **por más que el atacante sepa qué técnica estamos usando**.

Ej: crackear un key AES-256 usando 10^38 supoercomputadoras Tianhe-2 (la 4ta más potente al 2019), tomaría más tiempo que la edad actual del universo. Y eso son muchísimas veces más que una supercomputadora por cada grano de arena en la Tierra.

Los buenos algoritmos y prácticas de seguridad, son **abiertos**, estudiados y probados públicamente.

Y por eso también es mala tu solución casera: no tiene cientos de expertos probándola validándola. Y la oscuridad no te va a salvar de alguien decidido :)

Encriptación simétrica vs asimétrica
====================

Simétrica
-----------

A y B usan una misma clave, única, compartida, para encriptar y desencriptar sus mensajes. Ninguna otra clave funciona.

- Proceso simple
- Problema: tenemos que ponernos de acuerdo en la clave, que es un secreto pero tenemos que comunicarlo en algún momento (alguien puede atacar ese proceso).
- Alguien que accede a la clave, tiene conocimiento absoluto, y puede falsificar mensajes también.


Asimétrica
------------

Cada quien tiene un par de claves, una privada y una pública. Y el algoritmo garantiza que:

- Algo encriptado con una clave pública, solo puede ser desencriptado con su clave privada correspondiente.
- Algo encriptado con una clave privada, solo puede ser desencriptado con su clave pública correspondiente.

- Proceso bastante más complejo
- Pero las claves privadas nunca se comparten, la parte secreta nunca se transmite.
- Igual tenemos que confiar en que conocemos la clave pública de alguien (pero no es un secreto!).

Extra  (no se evalúa):
----------------------------

Cómo mandamos un mensaje con encriptación asimétrica?

Si A quiere mandarle algo a B, lo **encripta** dos veces (como una caja adentro de otra caja):
    
- Con la clave pública de B: entonces solo B puede desencriptar el mensaje (usando su privada).
- Y con su clave privada de A: entonces solo la clave pública de A va a poder desencriptarlo, lo que le asegura a B que el mensaje fue realmente escrito por A.

Consecuencia: solo B puede leer el mensaje, y está seguro de que A fue el autor.

Prácticas y tecnologías específicas
====================

Guardado de passwords
====================

Cómo guardarían una password?

"La encripto!"

![](./no.jpg)

Las passwords **no se encryptan!!!**

Las passwords se **hasheah**.

Un algoritmo de encriptación es de "ida y vuelta": puedo desencriptar el dato encriptado, usando la clave.

**No** quiero que alguien que entró al servidor, pueda desencriptar. Si mi código puede desencriptar la password, el atacante dentro de mi servidor también.


Un algoritmo de hashing es de "solo ida": entra el dato, sale un hash. Distintos datos dan distintos hashes. Pero a partir de un hash, **no** puedo obtener el dato original. No hay forma.

Un atacante que entró a mi server, va a poder ver los hashes, pero sigue sin poder obtener las passwords.

El dato de entrada para el algoritmo de hashing es más que solo la password. Normalmente se usa una semilla y data extra.

HTTPS
=======

Por default, las requests y responses de HTTP son texto que viaja en paquetes de TCP.

Esto genera dos problemas gigantes:

**Privacidad**: Cualquier intermediario haciendo de pasamanos del paquete, puede ver su contenido! Y cualquier compañero de red wifi puede ver todos los paquetes que se están pasando por la red también!

**Autenticidad**: Cualquier intermediario podría retener la request, armar una response, y devolvérmela. Cómo se si fue Google el que me dio esta response, o fue Fibertel metiéndose en el medio? Le puedo dar data privada?

Solución: **HTTPS**.

**Privacidad**: HTTPS **encripta** el contenido de las requests y responses, por lo que los paquetes de red van a ser ilegibles para cualquiera que no sea el origen o destino de ellos.

**Autenticidad**: HTTPS introduce un mecanismo de **certificados** que permiten validar si una response vino del dominio que dice venir. Un certificado no es más que un par de claves pública y privada, asociadas a un dominio en particular, que se usan para firmar las responses. Y además, existen entidades que certifican las claves públicas asociadas a cada dominio.

- Requiere tener los certificados de estas entidades en nuestra PC, guardados (los tiene el sistema operativo, se actualizan).
- Los certificados vencen, hay que renovarlos.

Problemas de HTTPS
-------------------------------

- ~~Costo del certificado~~ Ya no! (Lets Encrypt)[https://letsencrypt.org/] nos lo provee gratis y automatizado.
- Costo de procesamiento: firmar, encriptar y desencriptar son cosas pesadas.

Podemos usarlo en algunas partes. Pero por lo general, es mejor asumir el costo y usar HTTPS en todo.

Ataque: SQL injections
===================

Imaginemos un formulario para buscar contactos por email en nuestro sitio:

```
  email: _____________
```


Código:


```python
def buscar_contacto():
    query = "SELECT * FROM usuarios WHERE email = '" + form.email + "'"
    resultados = db.run(query)
    return resultados
    
```        

Qué pasa si el usuario rellena el form así?:

```
  email: bla' OR 'hola' = 'hola
```

Qué usuarios aparecerían en los resultados de la búsqueda?


Y si lo rellena así?:

```
  email: bla'; DROP TABLE usuarios; SELECT 'hola
```

O así?:

```
  email: bla'; UPDATE usuarios SET admin = True WHERE email = 'fisadev@gmail.com'; SELECT 'hola
```

Solución
---------

**NUNCA** concatenen input de usuarios con SQL.

Usen **consultas preparadas**! Todos los motores de bases de datos lo soportan.

```python
def buscar_contacto():
    query = "SELECT * FROM usuarios WHERE email = ?"
    resultados = db.run(query, form.email)
    return resultados
```        

El motor sabe que lo que venga en los parámetros **no es ejecutable**, sino solo un valor (en este caso, lo trataría como un texto entero).

Ataque: XSS (cross-site scripting)
=============================

Imaginemos que mostramos los comentarios de nuestros usuarios así:

```html
<h3>{{ comentario.usuario.nombre }}: {{ comentario.fecha }}</h3>
<p>{{ comentario.texto }}</p>
```


Qué pasa si un usuario postea este comentario?:

```
   Hola!
   <script>
       while (True) {
           alert("aguante river!");
       }
   </script>
```


O peor: puede estar por ejemplo extrayendo datos, o redireccionando a otro sitio, o impidiendo que el usuario haga algo, etc.

Solución
------

Todos los frameworks tienen alguna herramienta para **sanitizar input de usuarios** al mostrarlo dentro del HTML. 

Muchos lo hacen **por default** (Django por ejemplo), y pueden desactivarlo sin confían en algún input en particular.

**No** intenten sanitizar a mano. Es un problema complejo, usen herramientas ya probadas por la comunidad.

Ataque: XSRF o CSRF (cross-site request forgery)
=============================

Imaginemos que en nuestro sitio permitimos que los usuarios posteen links en sus comentarios, poniendo la url y texto que quieran.

Qué sucedería si un atacante hace esto?:

1. Postea un link con texto "oferta de espadas gratis!", que apunta a un sitio del atacante.
2. El sitio del atacante muestra un form que pide "cuántas espadas querés gratis?", pero además tiene algunos campos ocultos con otros datos, y es un form que hace POST a la url `nuestrositio.com/postear_comentario`.

Qué va a suceder cuando la gente haga click en su comentario, rellene ese form, y haga click en el submit del form?

**No** queremos que cualquiera pueda decirle al navegador "hacé este POST al sitio, usando la cookie de sesión del usuario", sin que el usuario haya pedido un form y posteado él mismo.

Queremos que solo se pueda postear cuando el usuario realmente estaba viendo un form nuestro!

Solución
------

Siempre que generamos un form para que el usuario postee, creamos junto un **token**, lo incluímos en el form y lo guardamos también en la db como "pendiente de postear".

Cuando el usuario postea nos llega su request con el token dentro del form: **validamos el token** y lo borramos de la db.

Si nos llega un post sin token, o con un token ya usado, **rechazamos** al request, es un atacante!

No facilitar la vida a atacantes
=============================

Todo el software que usemos, **va a tener vulnerabilidades**: sistema operativo, server web, base de datos, lenguaje de programación, frameworks y libs externas, etc.

Esas vulnerabilidades se **publican** cuando se descubren. Es muy fácil ir a ver la lista de vulnerabilidades conocidas de la mayoría de las cosas.

Por eso es muy fácil para un atacante aprovechar las vulnerabilidades conocidas, si **sabe qué software estamos usando**.

Por lo general es buena idea **dar la menor cantidad de información posible.**

Que las requests no digan qué server y versión usamos. Que el html no diga qué framework y versión lo generó. Que si ocurre un error, no se le muestre al usuario la página de debug con toooda la info interna de la aplicación. Etc.

"No es seguridad por oscuridad?????"

No. Porque nuestra seguridad no se "basa" solo en que eso no se sepa. Solo estamos **complicando** un poco más lo que un atacante tiene que hacer (va a tener que probar más cosas, adivinar, etc).

Almacenamiento de secrets
=============================

Nuestro código va a tener que usar data secreta:

- Tokens de OAuth
- Semillas para los algoritmos de hashing
- Usuario y password de la base de datos
- Usuario y password de la cuenta de mail usada para los correos del sitio
- ...

Pero si alguien logra acceder al repo de código? O si el código lo estamos publicando como software libre??

O si no queremos que los programadores puedan acceder a la data privada de los usuarios?? (ej: google y tus emails)

Por lo general, **no** es buena idea que esos datos secretos importantes estén guardados en el código.

Solución
------

Hay herramientas específicas para administrar secrets en producción (especies de bases de datos para esa data sensible).

Nuestro código usa valores por default falsos, que andan solo en desarrollo, pero intenta obtener datos del entorno de producción para reemplazarlos.

Cuando corre en local, no ve esos datos, usa los valores por default.

Cuando corre en producción, tiene acceso a esos datos, y usa esos valores reales.

Ejemplo en Python y Django:

```python
import django_heroku

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}

django_heroku.settings(locals())
```

En mi notebook de desarrollo, la base de datos es un archivo sqlite. En producción, la función `django_heroku.settings()` lee los datos de conexión reales de Heroku, y reemplaza el valor de la setting `DATABASES`.